In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

**Data Preprocessing Based on Luke's Result**

In [3]:
f_data = pd.read_csv('no_dup_data.csv')

In [4]:
f_data.head(10)

,Unnamed: 0,customer_id,account_id,ed_id,event_name,Date,Time,journey_steps_until_end
0,0,-784961211,1773350293,12,application_web_approved,2023-03-22,08:45:22,1
1,1,-784961211,1773350293,19,application_web_view,2023-03-22,13:32:10,2
2,14,-784961211,1773350293,3,application_web_submit,2023-03-22,13:32:10,3
3,15,-784961211,1773350293,2,campaign_click,2023-03-22,14:45:22,4
4,16,-784961211,1773350293,19,application_web_view,2023-07-27,14:57:56,5
5,21,-784961211,1773350293,19,application_web_view,2023-08-29,16:01:06,6
6,24,15849251,383997507,4,browse_products,2021-11-04,14:11:15,1
7,25,15849251,383997507,4,browse_products,2021-11-04,14:11:29,2
8,26,15849251,383997507,4,browse_products,2021-11-04,14:12:10,3
9,27,15849251,383997507,4,browse_products,2021-11-04,14:12:21,4


In [42]:
f_data.shape

(55853910, 8)

In [43]:
account_ids = f_data['account_id'].unique()
account_ids.shape

(1735457,)

In [44]:
def bagging_func(n, account_ids):
    sub_account_id = np.random.choice(account_ids, size=n, replace=False)
    sample_f_data = f_data[f_data['account_id'].isin(sub_account_id)]

    return sample_f_data

In [45]:
# take part of the dataset
sample_f_data = bagging_func(1000, account_ids)
sample_f_data.head(10)

,Unnamed: 0,customer_id,account_id,ed_id,event_name,Date,Time,journey_steps_until_end
23742,29441,2066488031,-4669228,29,account_activitation,2021-03-22,00:00:00,1
23743,29442,2066488031,-4669228,22,pre-application_(3rd_party_affiliates),2021-03-22,06:00:00,2
23744,29443,2066488031,-4669228,2,campaign_click,2021-03-22,06:00:00,3
23745,29444,2066488031,-4669228,19,application_web_view,2021-03-22,15:00:53,4
23746,29445,2066488031,-4669228,19,application_web_view,2021-03-22,15:00:54,5
23747,29447,2066488031,-4669228,19,application_web_view,2021-03-22,15:01:20,6
23748,29448,2066488031,-4669228,3,application_web_submit,2021-03-22,15:01:23,7
23749,29449,2066488031,-4669228,19,application_web_view,2021-03-22,15:01:23,8
23750,29450,2066488031,-4669228,19,application_web_view,2021-03-22,15:01:24,9
23751,29451,2066488031,-4669228,19,application_web_view,2021-03-22,15:01:28,10


In [46]:
sample_f_data.account_id.unique().shape

(1000,)

In [47]:
sample_f_data = sample_f_data[['customer_id', 'account_id', 'ed_id', 'journey_steps_until_end']]

In [48]:
#sample_f_data.to_csv('sample_data.csv')

In [49]:
sample_f_data.head(5)

,customer_id,account_id,ed_id,journey_steps_until_end
23742,2066488031,-4669228,29,1
23743,2066488031,-4669228,22,2
23744,2066488031,-4669228,2,3
23745,2066488031,-4669228,19,4
23746,2066488031,-4669228,19,5


In [50]:
# getting all journeys for each account
ed_id_lists = sample_f_data.groupby(['account_id'])['ed_id'].apply(list).reset_index()

max_length = ed_id_lists['ed_id'].apply(len).max()
ed_id_lists['ed_id'] = ed_id_lists['ed_id'].apply(lambda x: x + [0] * (max_length - len(x)))

ed_id_lists

,account_id,ed_id
0,-2141104908,"[2, 12, 4, 4, 4, 4, 4, 4, 4, 24, 4, 4, 4, 4, 4..."
1,-2133280906,"[2, 12, 4, 11, 5, 6, 1, 1, 1, 1, 21, 1, 0, 0, ..."
2,-2123679930,"[12, 24, 1, 1, 21, 21, 24, 1, 1, 1, 21, 24, 1,..."
3,-2120142849,"[12, 4, 4, 11, 1, 5, 4, 4, 11, 5, 5, 4, 11, 5,..."
4,-2096763041,"[19, 19, 19, 19, 19, 19, 3, 19, 19, 12, 4, 11,..."
...,...,...
995,2127297491,"[6, 19, 19, 3, 19, 19, 3, 19, 19, 12, 19, 6, 6..."
996,2128409897,"[12, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
997,2140113580,"[24, 24, 24, 2, 4, 19, 19, 3, 19, 19, 3, 19, 1..."
998,2146012064,"[4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 11, 1,..."


In [51]:
# the code of Luke, possible future modifications
def event_label(x):
    act_list = np.array([29,12,15])
    ord_list = np.array([7,18])

    is_act = False
    is_ord = False

    # parse and get condition checks
    # if any activated, 
    if any(np.in1d(x,act_list)):
        is_act = True
    if any(np.in1d(x,ord_list)):
        is_ord = True

    # activated only
    if is_act == True and is_ord == False:
        return "Activated, No Order"

    # ordered only
    elif is_act == False and is_ord == True:
        return "Ordered, Not Activated"

    # activated and ordered
    elif is_act == True and is_ord == True:
        return "Activated and Ordered"
    
    # accounts neither fit
    else:
        return "Neither"

# vectorize function
event_label_vec = np.vectorize(event_label)

In [52]:
ed_id_lists["customer_label"] = event_label_vec(ed_id_lists["ed_id"])

In [53]:
# orginal customer_labels
ed_id_lists.head()

,account_id,ed_id,customer_label
0,-2141104908,"[2, 12, 4, 4, 4, 4, 4, 4, 4, 24, 4, 4, 4, 4, 4...","Activated, No Order"
1,-2133280906,"[2, 12, 4, 11, 5, 6, 1, 1, 1, 1, 21, 1, 0, 0, ...","Activated, No Order"
2,-2123679930,"[12, 24, 1, 1, 21, 21, 24, 1, 1, 1, 21, 24, 1,...","Activated, No Order"
3,-2120142849,"[12, 4, 4, 11, 1, 5, 4, 4, 11, 5, 5, 4, 11, 5,...","Activated, No Order"
4,-2096763041,"[19, 19, 19, 19, 19, 19, 3, 19, 19, 12, 4, 11,...","Activated, No Order"


In [54]:
necessary_labels = ["Activated, No Order", "Activated and Ordered"]

In [55]:
ed_id_lists = ed_id_lists[ed_id_lists['customer_label'].isin(necessary_labels)]

In [56]:
ed_id_lists.customer_label.unique()

array(['Activated, No Order', 'Activated and Ordered'], dtype=object)

In [57]:
ed_id_lists['customer_label'].describe()

count                    1000
unique                      2
top       Activated, No Order
freq                      788
Name: customer_label, dtype: object

In [58]:
# turning `customer_label` to true labels
codes, uniques = pd.factorize(ed_id_lists['customer_label'])
ed_id_lists['customer_label'] = codes
ed_id_lists.shape

(1000, 3)

In [59]:
# activated & ordered = 0, activated, no order = 1
ed_id_lists.head(10)

,account_id,ed_id,customer_label
0,-2141104908,"[2, 12, 4, 4, 4, 4, 4, 4, 4, 24, 4, 4, 4, 4, 4...",0
1,-2133280906,"[2, 12, 4, 11, 5, 6, 1, 1, 1, 1, 21, 1, 0, 0, ...",0
2,-2123679930,"[12, 24, 1, 1, 21, 21, 24, 1, 1, 1, 21, 24, 1,...",0
3,-2120142849,"[12, 4, 4, 11, 1, 5, 4, 4, 11, 5, 5, 4, 11, 5,...",0
4,-2096763041,"[19, 19, 19, 19, 19, 19, 3, 19, 19, 12, 4, 11,...",0
5,-2090443149,"[12, 19, 3, 4, 1, 4, 4, 24, 0, 0, 0, 0, 0, 0, ...",0
6,-2086016383,"[2, 19, 19, 19, 19, 19, 19, 3, 12, 4, 4, 4, 4,...",0
7,-2070876890,"[12, 2, 4, 4, 24, 1, 21, 4, 11, 1, 5, 6, 5, 4,...",0
8,-2067325994,"[12, 3, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0
9,-2065030482,"[19, 19, 3, 2, 19, 19, 3, 19, 19, 19, 19, 19, ...",0


**Building RNN Model**

In [60]:
# adding masks for each observation
X = ed_id_lists['ed_id']  # Features
X = X.values.tolist()
mask = [[1 if x > 0 else x for x in sublist] for sublist in X]
y = ed_id_lists['customer_label']  # Target variable
y = y.values.tolist()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, mask_train, mask_test = train_test_split(X, y, mask, test_size=0.2, random_state=42)

In [61]:
train_dset = torch.utils.data.TensorDataset(torch.tensor(X_train,
                                                         dtype=torch.long),
                                            torch.tensor(y_train,
                                                         dtype=torch.long),
                                            torch.tensor(mask_train,
                                                         dtype=torch.float))

test_dset = torch.utils.data.TensorDataset(torch.tensor(X_test,
                                                        dtype=torch.long),
                                          torch.tensor(y_test,
                                                        dtype=torch.long),
                                          torch.tensor(mask_test,
                                                        dtype=torch.float))

In [62]:
# getting data loader for training and predicting process
batch_size = 128

train_loader = torch.utils.data.DataLoader(train_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2
                         )

test_loader = torch.utils.data.DataLoader(test_dset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=2
                         )

In [63]:
# small test of correctness
x, y, m = next(iter(train_loader))
print(x.shape, y.shape, m.shape)

print(x)
print(x.shape)
print(y)
print(y.shape)
print(m)
print(m.shape)

torch.Size([128, 438]) torch.Size([128]) torch.Size([128, 438])
tensor([[12,  4,  4,  ...,  0,  0,  0],
        [19, 19,  3,  ...,  0,  0,  0],
        [21,  2, 12,  ...,  0,  0,  0],
        ...,
        [12,  4,  4,  ...,  0,  0,  0],
        [12,  1,  4,  ...,  0,  0,  0],
        [12,  2,  1,  ...,  0,  0,  0]])
torch.Size([128, 438])
tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([128])
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [65]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, \
                 num_rec_layers=1, rec_layer=nn.RNN):
        super(RNNClassifier, self).__init__()
        # define all layers we need,
        # their parameters will be initialized automatically

        # nn.Embedding layer turns input sentences into word embeddings
        # with input and output dimension given by vocab_size and embedding_dim
        # self.word_embeddings = nn.Embedding(embedding_dim)

        # depending on the value of num_rec_layers, the corresponding number
        # of rec_layers (either RNN or LSTM) with batch_first=True and hidden
        # dimension given by hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.num_rec_layers = num_rec_layers
        self.rnn1 = rec_layer(embedding_dim, hidden_dim, batch_first=True)
        if self.num_rec_layers == 2:
            self.rnn2 = rec_layer(hidden_dim, hidden_dim, batch_first=True)

        # a final linear layer with sigmoid activation with input and output
        # dimension given by hidden_dim and 1.
        self.sigmoid = nn.Sigmoid()
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, sentences, mask):
        # sentences shape: [B, L], mask shape: [B, L]
        # embedding = word_embeddings(embedding_dim)# call your embedding layer, output shape: [B, L, DE]
        embedding = self.word_embeddings(sentences)
        out, hidden = self.rnn1(embedding)# call your rnn1, output shape: [B, L, DH]
        if self.num_rec_layers == 2:
            out, hidden = self.rnn2(out, hidden)# call your rnn2, output shape: [B, L, DH]
        out = (out*mask[:, :, None]).mean(dim=1) # shape: [B, DH]
        res = self.sigmoid(self.hidden2label(out))# call your hidden2label, output shape: [B, 1]
        return res

In [66]:
# create a particular instance of the model, do a 1 layer vaniila RNN with embedding_dim=hidden_dim=128
rnn = RNNClassifier(128, 128, 500)
rnn.to(device)

RNNClassifier(
  (word_embeddings): Embedding(500, 128)
  (rnn1): RNN(128, 128, batch_first=True)
  (sigmoid): Sigmoid()
  (hidden2label): Linear(in_features=128, out_features=1, bias=True)
)

In [67]:
# demo forward pass with the mini-batch that we generated above
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = x.to(device)
y = y.to(device)
m = m.to(device)

y_pred = rnn(x, m)
print(y_pred.shape)
print(y_pred[:10])

torch.Size([128, 1])
tensor([[0.5130],
        [0.5159],
        [0.5147],
        [0.5127],
        [0.5092],
        [0.5157],
        [0.5150],
        [0.5130],
        [0.5159],
        [0.5157]], grad_fn=<SliceBackward0>)


In [68]:
learning_rate = 0.001
optimizer =optim.RMSprop(rnn.parameters(), lr=learning_rate)

lossfun = nn.BCELoss(reduction='mean')

In [69]:
def train_epoch(train_loader, model, lossfun, optimizer, device):
    model.train()
    # iterate over mini-batches
    for it, (inputs, labels, mask) in enumerate(train_loader):
        # move everything to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        mask = mask.to(device)

        model.zero_grad()
        # forward pass
        output = model(inputs, mask)

        loss = lossfun(output.view(-1), labels.float())
        # backward pass
        loss.backward()

        # update model parameters
        optimizer.step()

def evaluate(loader, model, lossfun, device):
    model.eval()
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    # iterate over mini-batches
    for it, (inputs, labels, mask) in enumerate(loader):
        # move everything to the device
        inputs = inputs.to(device)
        labels = labels.to(device)
        mask = mask.to(device)

        # forward pass
        output = model(inputs, mask)

        # calculate loss value
        loss = lossfun(output.view(-1), labels.float())
        total_loss += loss.item()

        # calculate test accuracy
        pred = output.view(-1) > 0.5
        correct = (pred == labels.bool())
        total_acc += torch.sum(correct).item() / len(correct)

    total = it + 1
    return total_loss / total, total_acc / total


def train(train_loader, test_loader, model, lossfun, optimizer, \
          device, num_epochs):
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    # irerate over training epochs
    for epoch in range(num_epochs):
        # at each step, we do a training epoch and evaluate on train and test data
        train_epoch(train_loader, model, lossfun, optimizer, device)
        train_loss, train_acc = evaluate(train_loader, model, lossfun, device)
        train_loss_.append(train_loss)
        train_acc_.append(train_acc)
        test_loss, test_acc = evaluate(test_loader, model, lossfun, device)
        test_loss_.append(test_loss)
        test_acc_.append(test_acc)

        print(f'Epoch: {epoch+1:3d}/{num_epochs:3d} '
              f'Training Loss: {train_loss_[epoch]:.3f}, Testing Loss: {test_loss_[epoch]:.3f}, '
              f'Training Acc: {train_acc_[epoch]:.3f}, Testing Acc: {test_acc_[epoch]:.3f}')

    return train_loss_, train_acc_, test_loss_, test_acc_

In [70]:
# let's first train a vanilla RNN
a, b, c, d = train(train_loader, test_loader, rnn, lossfun, \
                   optimizer, device, num_epochs=30)

Epoch:   1/ 30 Training Loss: 0.626, Testing Loss: 0.627, Training Acc: 0.779, Testing Acc: 0.793
Epoch:   2/ 30 Training Loss: 0.623, Testing Loss: 0.634, Training Acc: 0.817, Testing Acc: 0.802
Epoch:   3/ 30 Training Loss: 0.580, Testing Loss: 0.582, Training Acc: 0.836, Testing Acc: 0.836
Epoch:   4/ 30 Training Loss: 0.559, Testing Loss: 0.539, Training Acc: 0.849, Testing Acc: 0.852
Epoch:   5/ 30 Training Loss: 0.541, Testing Loss: 0.536, Training Acc: 0.872, Testing Acc: 0.882
Epoch:   6/ 30 Training Loss: 0.546, Testing Loss: 0.540, Training Acc: 0.839, Testing Acc: 0.854
Epoch:   7/ 30 Training Loss: 0.543, Testing Loss: 0.533, Training Acc: 0.856, Testing Acc: 0.883
Epoch:   8/ 30 Training Loss: 0.520, Testing Loss: 0.511, Training Acc: 0.862, Testing Acc: 0.882
Epoch:   9/ 30 Training Loss: 0.501, Testing Loss: 0.493, Training Acc: 0.866, Testing Acc: 0.881
Epoch:  10/ 30 Training Loss: 0.506, Testing Loss: 0.495, Training Acc: 0.856, Testing Acc: 0.869
Epoch:  11/ 30 Train

In [73]:
new_sample_f_data = bagging_func(1000, account_ids)

In [74]:
new_sample_f_data = new_sample_f_data[['customer_id', 'account_id', 'ed_id', 'journey_steps_until_end']]

In [77]:
# getting all journeys for each account
new_ed_id_lists = new_sample_f_data.groupby(['account_id'])['ed_id'].apply(list).reset_index()

max_length = new_ed_id_lists['ed_id'].apply(len).max()
new_ed_id_lists['ed_id'] = new_ed_id_lists['ed_id'].apply(lambda x: x + [0] * (max_length - len(x)))

new_ed_id_lists

,account_id,ed_id
0,-2137038361,"[4, 19, 19, 19, 3, 19, 12, 19, 4, 4, 11, 1, 24..."
1,-2133413622,"[15, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,-2131020712,"[12, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,..."
3,-2122327136,"[19, 19, 3, 19, 19, 19, 3, 19, 19, 19, 19, 3, ..."
4,-2120758693,"[19, 19, 19, 19, 19, 19, 19, 3, 19, 12, 4, 4, ..."
...,...,...
995,2136076029,"[2, 12, 19, 19, 19, 19, 19, 19, 4, 11, 4, 1, 4..."
996,2141067749,"[6, 19, 19, 19, 19, 19, 19, 19, 3, 19, 12, 6, ..."
997,2141840907,"[2, 22, 12, 1, 21, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
998,2143727441,"[12, 1, 4, 5, 11, 19, 3, 6, 2, 0, 0, 0, 0, 0, ..."


In [78]:
new_event_label_vec = np.vectorize(event_label)
new_ed_id_lists["customer_label"] = new_event_label_vec(new_ed_id_lists["ed_id"])

In [79]:
ed_id_lists.head()

,account_id,ed_id,customer_label
0,-2141104908,"[2, 12, 4, 4, 4, 4, 4, 4, 4, 24, 4, 4, 4, 4, 4...",0
1,-2133280906,"[2, 12, 4, 11, 5, 6, 1, 1, 1, 1, 21, 1, 0, 0, ...",0
2,-2123679930,"[12, 24, 1, 1, 21, 21, 24, 1, 1, 1, 21, 24, 1,...",0
3,-2120142849,"[12, 4, 4, 11, 1, 5, 4, 4, 11, 5, 5, 4, 11, 5,...",0
4,-2096763041,"[19, 19, 19, 19, 19, 19, 3, 19, 19, 12, 4, 11,...",0


In [80]:
necessary_labels = ["Activated, No Order", "Activated and Ordered"]

In [81]:
new_ed_id_lists = new_ed_id_lists[new_ed_id_lists['customer_label'].isin(necessary_labels)]

In [82]:
codes, uniques = pd.factorize(new_ed_id_lists['customer_label'])
new_ed_id_lists['customer_label'] = codes

In [83]:
new_ed_id_lists.head(10)

,account_id,ed_id,customer_label
0,-2137038361,"[4, 19, 19, 19, 3, 19, 12, 19, 4, 4, 11, 1, 24...",0
1,-2133413622,"[15, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1
2,-2131020712,"[12, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,...",1
3,-2122327136,"[19, 19, 3, 19, 19, 19, 3, 19, 19, 19, 19, 3, ...",0
4,-2120758693,"[19, 19, 19, 19, 19, 19, 19, 3, 19, 12, 4, 4, ...",0
5,-2119498841,"[12, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2,...",0
6,-2115288757,"[12, 2, 3, 4, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0
7,-2114975114,"[12, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2,...",1
8,-2113651508,"[2, 22, 12, 1, 1, 21, 1, 21, 1, 1, 21, 0, 0, 0...",0
9,-2109691753,"[2, 12, 4, 11, 5, 6, 1, 4, 11, 5, 6, 7, 29, 0,...",1


In [84]:
new_X = new_ed_id_lists['ed_id']  # Features
new_X = new_X.values.tolist()
new_mask = [[1 if x > 0 else x for x in sublist] for sublist in new_X]
new_y = new_ed_id_lists['customer_label']  # Target variable
new_y = new_y.values.tolist()

In [88]:
# need to get the data from Luke and then perform testing part
test_dataset = torch.utils.data.TensorDataset(torch.tensor(new_X,
                                                        dtype=torch.long),
                                          torch.tensor(new_y,
                                                        dtype=torch.long),
                                          torch.tensor(new_mask,
                                                        dtype=torch.float))

In [89]:
new_import_test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=2
                         )

In [91]:
test_loss, test_acc = evaluate(new_import_test_loader, rnn, lossfun, device)
print(test_loss)
print(test_acc)

0.37640491873025894
0.8780799278846154
